In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf


In [2]:
train_data = pd.read_csv("/content/drive/MyDrive/Dataset/train_data.csv")
test_data = pd.read_csv("/content/drive/MyDrive/Dataset/test_data.csv")
validation_data = pd.read_csv("/content/drive/MyDrive/Dataset/validation_data.csv")


In [3]:
train_data.head()

,combined_text,target
0,tle protrump rocker tednugent fires back at co...,0
1,tle trump speaks with palestinian leader abbas...,1
2,tle sanders blasts misogynist bernie bro fans ...,0
3,tle no joke hillary clinton tries to censor la...,0
4,tle as trump visits un new york prepares for s...,1


In [4]:
tk = tf.keras.preprocessing.text.Tokenizer(num_words=5000,oov_token="<oov>")

In [5]:
text=list(train_data["combined_text"])

In [6]:
tk.fit_on_texts(text)

In [7]:
import pickle
with open("/content/drive/MyDrive/Dataset/tokenizer_ann.pickle","wb") as fp:
  pickle.dump(tk,fp,protocol=pickle.HIGHEST_PROTOCOL)

In [8]:
X_train = train_data["combined_text"]
X_test = test_data["combined_text"]
X_val = validation_data["combined_text"]

y_train = train_data["target"]
y_test = test_data["target"]
y_val = validation_data["target"]

In [9]:
X_train_seq = tk.texts_to_sequences(X_train)
X_test_seq = tk.texts_to_sequences(X_test)
X_val_seq = tk.texts_to_sequences(X_val)

X_train_seq_trunc = tf.keras.preprocessing.sequence.pad_sequences(X_train_seq, maxlen=100)
X_test_seq_trunc = tf.keras.preprocessing.sequence.pad_sequences(X_test_seq, maxlen=100)
X_val_seq_trunc = tf.keras.preprocessing.sequence.pad_sequences(X_val_seq, maxlen=100)

In [10]:
from tensorflow.keras import layers,models

In [11]:
emb_model = models.Sequential()
emb_model.add(layers.Embedding(tk.num_words,8, input_length=100))
emb_model.add(layers.Flatten())
emb_model.add(layers.Dense(1, activation='sigmoid'))

In [12]:
emb_model.compile(optimizer='adam',loss=tf.keras.losses.BinaryCrossentropy(),metrics=[tf.keras.metrics.BinaryAccuracy()])
emb_model.fit(x=X_train_seq_trunc,y=y_train,batch_size=256,epochs=3,validation_data=(X_val_seq_trunc,y_val))

Epoch 1/3
159/159 [==============================] - 2s 5ms/step - loss: 0.4869 - binary_accuracy: 0.8010 - val_loss: 0.2279 - val_binary_accuracy: 0.9245
Epoch 2/3
159/159 [==============================] - 1s 4ms/step - loss: 0.1585 - binary_accuracy: 0.9504 - val_loss: 0.1134 - val_binary_accuracy: 0.9667
Epoch 3/3
159/159 [==============================] - 1s 4ms/step - loss: 0.0924 - binary_accuracy: 0.9729 - val_loss: 0.0769 - val_binary_accuracy: 0.9789


In [13]:
emb_model.evaluate(X_test_seq_trunc,y_test)

71/71 [==============================] - 0s 1ms/step - loss: 0.0900 - binary_accuracy: 0.9688


[0.09002187848091125, 0.9688196182250977]

In [14]:
emb_model.save("/content/drive/MyDrive/Dataset/ann_model.hdf5")